# 🧠 cloudChronicles Lab #001: Disaster Recovery Detective

**Lab Type:** Idea 
**Estimated Time:** 30–45 mins 
**Skill Level:** Beginner

In [ ]:
# Let's begin by printing your name to personalize the notebook
your_name = "[YOUR NAME HERE]"
print(f"Welcome to the lab, {your_name}!")

## 🔍 STAR Method Lab Prompt

**Situation:** 
You are a Cloud Engineer at "GlobalConnect Inc.," a rapidly growing tech company whose core business relies heavily on a **mission-critical web application**. This application, currently hosted in a single GCP region (`us-central1`), serves millions of users daily. Recently, there was a minor outage in a related service, highlighting the urgent need for a robust **Disaster Recovery (DR) plan** to ensure continuous availability and minimize downtime in the event of a regional failure or significant incident.

**Task:** 
Your task is to design a **multi-region disaster recovery strategy** for GlobalConnect Inc.'s critical web application. This plan should ensure that the application can **fail over** to a secondary region with minimal data loss and downtime, maintaining business continuity even during a catastrophic event in the primary region. You need to identify the key GCP services to achieve this and outline their roles within your DR strategy.

**Action:** 
1.  **Choose a secondary GCP region** for your DR setup that is geographically distinct from `us-central1`. Consider network latency and regulatory requirements (though for this lab, a simple choice is fine).
2.  **Identify core components** of a typical web application (e.g., front-end, back-end compute, database, storage, load balancing).
3.  For each core component, **select appropriate GCP services** that support multi-region deployment and disaster recovery capabilities.
4.  **Outline the mechanism for data replication** between the primary and secondary regions for your chosen database and storage services.
5.  **Describe the failover process**: How will traffic be redirected to the secondary region during a disaster? What triggers this failover?
6.  **Consider backup and restore strategies**: How will data be backed up, and how can it be restored if needed?
7.  **Briefly explain how you would test this DR plan** (without actually performing the test).

**Expected Result:** 
A **concise Disaster Recovery Plan summary** in Markdown format, outlining the chosen GCP services, the multi-region strategy, data replication methods, failover mechanisms, and backup considerations. The summary should clearly articulate how each component contributes to a resilient and highly available application, similar to the example provided in the "Your Assignment" section below.

## ✍️ Your Assignment

_Use this section to complete your deliverable:_

```markdown
**GlobalConnect Inc. - Web Application Disaster Recovery Plan**

- **Primary Region**: `us-central1`
- **Secondary Region**: [YOUR CHOSEN SECONDARY REGION]

- **Application Components & GCP Services for DR:**
    - **Compute (e.g., Web Servers/APIs):** [GCP Service e.g., Compute Engine, GKE, App Engine]
        - *DR Strategy:* [e.g., Multi-region deployment, Instance templates, Managed Instance Groups]
    - **Database:** [GCP Service e.g., Cloud SQL, Cloud Spanner, Firestore]
        - *Data Replication:* [e.g., Cross-region replication, RPO/RTO considerations]
    - **Storage (e.g., User Uploads/Static Assets):** [GCP Service e.g., Cloud Storage]
        - *Data Replication/Resilience:* [e.g., Multi-regional bucket, Object Versioning]
    - **Load Balancing/Traffic Management:** [GCP Service e.g., Global External Load Balancer]
        - *Failover Mechanism:* [e.g., Health checks, Backend services across regions]

- **Failover Trigger & Process:**
    - [How is a disaster detected? How is traffic redirected?]

- **Backup & Restore Strategy:**
    - [What services are backed up? How frequently? Where are backups stored?]

- **DR Plan Testing:**
    - [Briefly describe how you would test this plan.]

```